# Investment Dashboard 



This is a python data process exercise following [Vincent Tatan's blog at Medium](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca).

The goal of this application is to help you retrieve and display the right financial insights quickly about a certain company.

------------------

### Main process :

1. Get stock ticker and name to be data source of dropdown control
2. Get stock price history from yahoo through pandas datareader when user choose a stock from dropdown control
3. Plot stock k chart , sma  , macd , rsi lines
4. Get stock financial information from financial website to do basic analysis and infer future value for trading decision.



### Parsing fincial information from MarketWatch web site
    For example, to get Google's financial information from this url : https://www.marketwatch.com/investing/stock/GOOG/financials



### Use pandas datareader through yahoo to get share price history
```
    pdr.DataReader(
            ticker.strip(),data_source='yahoo',start=dt(2020,1,1),end=end_date
    )
```


### Warning Signs List based on value investing logic ###

Given list of the companies, find out the feasibility to invest

1. Been in market minimal 10 years
1. Have the track records (EPS per year)
1. Have efficiency (ROE > 15%) — Net income / shareholder equity
1. Determine manipulation (ROA > 7%) — Net income / Total Asset
1. Have small long term debt (Long term debt <5* total income)
1. Low Debt to Equity
1. Ability to pay interest: (Interest Coverage Ratio >3) — EBIT / Interest expenses



### Decision rules for stock transaction

Decision making from each company in terms of return rate given the value investing methodology :

1. Find EPS Annual Compounded Growth Rate
2. Estimate EPS 10 years from now
3. Estimate stock price 10 years from now (Stock Price EPS * Average PE)
4. Determine target by price today based on returns(discount rate 15%/20%)
   - Discount rate indicated predicted inflation over the period of 5 years. 
5. Add margin of safety (Safety net 15%)
   - Margin calculation rate is the toleration for error. You add on additional safety net in the case of calculation error. The more the margin calculation rate, the more risk averse you are or vice versa.
6. Buy if market price is lower than the marginal price
7. Sell if market price is higher than the marginal price

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.offline as plyo
import numpy as np
import numpy_financial as npf
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as dash_html
import dash.dependencies as dsah_depend
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
import dash_bootstrap_components as dbc
import dashboard 
from dashboard import *
import stock_info_service 
from stock_info_service import *
import stock_info_service
from investment_analysis import *
import utils
import importlib
#importlib.reload(stock_info_service)
# from stock_info_service import *
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
cf.offline.go_offline() # with execute this command will cause QuantFig.iplot() run into error

In [15]:

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dashboard_layout()
app.title = 'Investment Dashboard'
register_callbacks(app)
# Run app and display result inline in the notebook
if __name__ == '__main__':
    app.run_server(debug=True,mode='inline',port=8123,dev_tools_ui=True,dev_tools_silence_routes_logging=False) # set mode='external' to use web browser

127.0.0.1 - - [07/Sep/2020 16:52:59] "GET /_shutdown_2c1f5273-dc7e-4fe5-b26f-5fadeca0d1bf HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 16:52:59] "GET /_alive_2c1f5273-dc7e-4fe5-b26f-5fadeca0d1bf HTTP/1.1" 200 -


In [10]:
!pip install dash-bootstrap-components

     |████████████████████████████████| 108 kB 516 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dash-bootstrap-components: filename=dash_bootstrap_components-0.10.5-py3-none-any.whl size=180067 sha256=023be51c6043bd5ebc1dd4baf3e771f621968f3e2554f9302a0a640aba6dc684
  Stored in directory: /Users/steven/Library/Caches/pip/wheels/6e/b9/51/b0d8d74d724a9775f0bb5b4bf6679d08f1ca5cd9e2dc182f70
Successfully built dash-bootstrap-components
You should consider upgrading via the '/Users/steven/GIT/ml/my_env/bin/python -m pip install --upgrade pip' command.


In [ ]:

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    global stockpricedf # Needed to modify global copy of stockpricedf
    print(selected_dropdown_value)
    stockpricedf = pdr.DataReader(
         selected_dropdown_value.strip(), data_source='yahoo',
         start=dt(2013, 1, 1), end=dt.now())
    return {
        'data': [{
             'x': stockpricedf.index,
             'y': stockpricedf.Close
    }]
}

In [5]:
df = get_stock_financial_info('GOOG.tew')

https://www.marketwatch.com/investing/stock/GOOG.tew/financials
      eps  epsgrowth  netincome  shareholderequity  roa  longtermdebt  \
2015    0          0          0                  0    0             0   
2016    0          0          0                  0    0             0   
2017    0          0          0                  0    0             0   
2018    0          0          0                  0    0             0   
2019    0          0          0                  0    0             0   

      interestexpense  ebitda  roe  interestcoverageratio  
2015                0       0  NaN                    NaN  
2016                0       0  NaN                    NaN  
2017                0       0  NaN                    NaN  
2018                0       0  NaN                    NaN  
2019                0       0  NaN                    NaN  


In [13]:
stockpricedf = get_stock_price('coke',start_date=dt(2013, 1, 1))

In [14]:
infer_reasonable_share_price('coke',df,stockpricedf,0.05,0.05,after_years=10)
	

1st year EPS 0.000000 
5th year EPS 0.000000
Annual Growth Rate nan
      eps  epsgrowth  netincome  shareholderequity  roa  longtermdebt  \
2015    0          0          0                  0    0             0   
2016    0          0          0                  0    0             0   
2017    0          0          0                  0    0             0   
2018    0          0          0                  0    0             0   
2019    0          0          0                  0    0             0   

      interestexpense  ebitda                  roe  interestcoverageratio  
2015                0       0                  nan                    nan  
2016                0       0                  nan                    nan  
2017                0       0                  nan                    nan  
2018                0       0                  nan                    nan  
2019                0       0                  nan                    nan  
PE ration inf


,annualgrowthrate,lasteps,futureeps,peratio,FV,PV,marginprice,lastshareprice,decision
ticker,,,,,,,,,
coke,NaN,0,NaN,inf,NaN,NaN,0,275.29,SELL
